In [1]:
from tensorflow.keras import callbacks, optimizers
from yolov4.tf import SaveWeightsCallback, YOLOv4
from yolov4.tflite import YOLOv4 as YOLOv4tflite
import time


In [2]:
#yolo = YOLOv4tflite(tiny=True)
#yolo.load_tflite("yolov4_416x550_fruits_20epochs.tflite")

yolo = YOLOv4(tiny=True)
yolo.classes = "../data/fruits-360/train-multiple_fruits_yolov4/_classes.txt"
yolo.input_size = (416, 544)
yolo.batch_size = 32
yolo.make_model()


In [3]:
train_data_set = yolo.load_dataset(
    "../data/fruits-360/train-multiple_fruits_yolov4/train_annotations.txt",
    image_path_prefix="../data/fruits-360/train-multiple_fruits_yolov4/train",
    label_smoothing=0.05
)
val_data_set = yolo.load_dataset(
    "../data/fruits-360/train-multiple_fruits_yolov4/valid_annotations.txt",
    image_path_prefix="../data/fruits-360/train-multiple_fruits_yolov4/valid",
    training=False
)

yolo.load_weights("yolov4-tiny-40epochs.weights", weights_type="yolo")

In [ ]:
epochs = 40
lr = 1e-6

optimizer = optimizers.Adam(learning_rate=lr)
yolo.compile(optimizer=optimizer, loss_iou_type="ciou")

def lr_scheduler(epoch):
    if epoch < int(epochs * 0.5):
        return lr
    if epoch < int(epochs * 0.8):
        return lr * 0.5
    if epoch < int(epochs * 0.9):
        return lr * 0.1
    return lr * 0.01

_callbacks = [
    callbacks.LearningRateScheduler(lr_scheduler),
    callbacks.TerminateOnNaN(),
    callbacks.TensorBoard(
        log_dir="yolov4_logs/logs",
    ),
    SaveWeightsCallback(
        yolo=yolo, dir_path="yolov4_logs/trained",
        weights_type="yolo", epoch_per_save=10
    ),
]

yolo.fit(
    train_data_set,
    epochs=epochs,
    callbacks=_callbacks,
    validation_data=val_data_set,
    validation_steps=25,
    validation_freq=5,
    steps_per_epoch=100,
)

Epoch 1/40
grid: 34*26 iou_loss: 4.97821 conf_loss: 9.04896 prob_loss: 23.0499363 total_loss 37.0771065
grid: 17*13 iou_loss: 4.67768145 conf_loss: 3.02762318 prob_loss: 17.8008957 total_loss 25.5061989
grid: 34*26 iou_loss: 6.48128843 conf_loss: 11.6896763 prob_loss: 28.5652504 total_loss 46.7362175
grid: 17*13 iou_loss: 4.71781445 conf_loss: 4.06968117 prob_loss: 18.8145466 total_loss 27.6020432
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
grid: 34*26 iou_loss: 3.385952 conf_loss: 5.86842299 prob_loss: 19.244978 total_loss 28.4993534
grid: 17*13 iou_loss: 3.0631671 conf_loss: 1.64668179 prob_loss: 15.038785 total_loss 19.7486343
grid: 34*26 iou_loss: 4.24677801 conf_loss: 9.13822937 prob_loss: 23.6104259 total_loss 36.9954338
grid: 17*13 iou_loss: 4.02237606 conf_loss: 4.77112722 prob_loss: 18.5598869 total_loss 27.3533897
grid: 34*26 iou_loss: 4.45701408 conf_loss: 8.74619865 prob_loss: 24.0820084 total_loss 37.2852249
grid: 17*13 iou_loss: 4.11870432 conf

grid: 17*13 iou_loss: 3.32010269 conf_loss: 2.05515623 prob_loss: 15.2924385 total_loss 20.6676979
grid: 34*26 iou_loss: 6.86970377 conf_loss: 13.7724791 prob_loss: 27.4335213 total_loss 48.0757065
grid: 17*13 iou_loss: 5.28321552 conf_loss: 4.64606428 prob_loss: 19.4568825 total_loss 29.3861618
grid: 34*26 iou_loss: 2.76684189 conf_loss: 5.99604225 prob_loss: 13.2461281 total_loss 22.0090122
grid: 17*13 iou_loss: 2.24005365 conf_loss: 1.42051697 prob_loss: 10.2821312 total_loss 13.9427013
grid: 34*26 iou_loss: 6.42264843 conf_loss: 11.6841259 prob_loss: 22.6263733 total_loss 40.7331505
grid: 17*13 iou_loss: 4.81540251 conf_loss: 3.77927017 prob_loss: 16.4637604 total_loss 25.0584335
grid: 34*26 iou_loss: 6.45427465 conf_loss: 9.65855122 prob_loss: 23.7830925 total_loss 39.895916
grid: 17*13 iou_loss: 3.90379071 conf_loss: 2.58831167 prob_loss: 14.4359436 total_loss 20.9280472
grid: 34*26 iou_loss: 3.22280931 conf_loss: 6.22612572 prob_loss: 17.3411484 total_loss 26.7900829
grid: 17*13

In [19]:
yolo.save_as_tflite("yolov4_416x550_fruits_80epochs.tflite")


INFO:tensorflow:Assets written to: /tmp/tmp60e9drax/assets


INFO:tensorflow:Assets written to: /tmp/tmp60e9drax/assets


In [20]:
yolo.save_dataset_for_mAP("yolov4_map/mAP-master", val_data_set)

In [17]:
yolo.inference(media_path="../data/fruits-360/test-multiple_fruits/apple_apricot_nectarine_peach_peach(flat)_pomegranate_pear_plum.jpg")

time: 67.79 ms
YOLOv4: Inference is finished


KeyboardInterrupt: 